In [93]:
import pandas as pd
from PIL import Image,ImageDraw,ImageFont
import os
import numpy as np
import math

In [94]:
ii = pd.read_csv("193398_individual_importances.csv")

In [95]:
ii.risk_1.value_counts().head()

ts_id_p1y_trafficstopsbysearchreason_consent_sum     299
dispatch_id_p1w_dispatchtype_domestic_disturb_sum    172
dispatch_id_p1w_dispatchtype_domestic_disturb_avg    145
dispatch_id_p5y_dispatchtype_assault_avg             124
irag_id_all_dayssincelastallegation_min              106
Name: risk_1, dtype: int64

In [96]:
ii.risk_2.value_counts().head()

dispatch_id_p1w_dispatchtype_domestic_disturb_avg    159
dispatch_id_p1w_dispatchtype_domestic_disturb_sum    137
dispatch_id_p5y_dispatchtype_assault_avg             116
irag_id_all_dayssincelastallegation_min               90
dispatch_id_p1y_dispatchtype_traffic_sum              76
Name: risk_2, dtype: int64

In [97]:
ii.risk_3.value_counts().head()

dispatch_id_p1w_dispatchtype_domestic_disturb_avg    74
dispatch_id_p1y_dispatchtype_traffic_sum             73
dispatch_id_p1y_dispatchtype_assist_sum              73
shifts_id_p5y_hourspershift_sum                      70
dispatch_id_p5y_dispatchtype_assault_avg             70
Name: risk_3, dtype: int64

In [98]:
ii.risk_4.value_counts().head()

dispatch_id_p1y_dispatchtype_traffic_sum     85
dispatch_id_p1y_dispatchtype_disorder_sum    76
dispatch_id_p1y_dispatchtype_drug_sum        74
dispatch_id_p1y_dispatchtype_shooting_sum    69
shifts_id_p5y_hourspershift_sum              66
Name: risk_4, dtype: int64

In [99]:
ii.risk_5.value_counts().head()

dispatch_id_p1y_dispatchtype_assist_sum             74
dispatch_id_p1y_dispatchtype_suspicious_sum         73
dispatch_id_p1y_dispatchtype_disorder_sum           70
dispatch_id_p1y_dispatchtype_intoxicated_avg        69
dispatch_id_p1w_dispatchinitiatiationtype_ci_sum    68
Name: risk_5, dtype: int64

In [100]:
ii.set_index("entity_id",inplace=True)

In [101]:
feature_table = pd.read_csv("193398_feattable.csv")
feature_table = feature_table[feature_table.feature_importance>0.00353]
feature_table.reset_index(drop=True,inplace=True)
top_feats = list(feature_table.feature)

In [102]:
len(top_feats)

10

In [103]:
top_feats

['ocag_id_all_officerage_max',
 'dispatch_id_p1y_dispatchinitiatiationtype_ci_sum',
 'dispatch_id_p1m_dispatchinitiatiationtype_ci_sum',
 'dispatch_id_p1m_dispatchtype_theft_sum',
 'dispatch_id_p1m_dispatchtype_domestic_disturb_sum',
 'dispatch_id_p1m_dispatchtype_disorder_sum',
 'dispatch_id_p1y_dispatchtype_assist_sum',
 'dispatch_id_p1y_dispatchtype_domestic_disturb_sum',
 'dispatch_id_p1y_dispatchtype_shooting_sum',
 'dispatch_id_p1y_dispatchtype_assault_sum']

In [104]:
singleton = 5570

In [105]:
singlevars = [
    ii.risk_1.loc[singleton],
    ii.risk_2.loc[singleton],
    ii.risk_3.loc[singleton],
    ii.risk_4.loc[singleton],
    ii.risk_5.loc[singleton]
    ]

In [106]:
singlevars

['shifts_id_p5y_hourspershift_sum',
 'dispatch_id_p5y_dispatchtype_assault_avg',
 'dispatch_id_p1y_dispatchinitiatiationtype_ci_sum',
 'dispatch_id_p1y_dispatchtype_traffic_sum',
 'dispatch_id_p1y_dispatchtype_assist_sum']

In [107]:
#hotvar = top_feats[9]
#cfeat = top_feats[9]
hotvar = singlevars[0]
cfeat = singlevars[0]

#### test matrix

In [108]:
test_matrix = pd.read_csv("193398_test_2014-04-01.csv")
test_matrix.set_index("officer_id",inplace=True)

In [109]:
test_matrix.sort_values(by='score',ascending=False,inplace=True)
labels = test_matrix.outcome
scores = test_matrix.score

In [110]:
len(test_matrix)

1466

In [111]:
test_matrix = test_matrix[test_matrix['ocag_id_all_officerage_max']!=0]

In [112]:
len(test_matrix)

1465

In [113]:
test_matrix['hotvar_imp'] = False

In [114]:
ii_idxs = [item for item in ii.index if item in test_matrix.index]

In [115]:
for i in ii_idxs:    
    featlist = [
        ii.risk_1.loc[i],
        ii.risk_2.loc[i],
        ii.risk_3.loc[i],
        ii.risk_4.loc[i],
        ii.risk_5.loc[i]
    ]
    
    if hotvar in featlist:
        test_matrix.hotvar_imp.loc[i] = True

In [116]:
print min(scores),max(scores)

0.0624336247543 0.145132730406


In [117]:
test_matrix.outcome.value_counts()

0    1309
1     156
Name: outcome, dtype: int64

In [118]:
tmp = test_matrix[['ocag_id_all_officerage_max','outcome']]

In [119]:
prior = float(156)/1309

In [120]:
max(tmp.ocag_id_all_officerage_max) - min(tmp.ocag_id_all_officerage_max)

45.0

In [121]:
test_matrix.hotvar_imp.value_counts()

False    1130
True      335
Name: hotvar_imp, dtype: int64

In [122]:
hotidxs = test_matrix.index[test_matrix.hotvar_imp==True]

## Binning

In [123]:
def binticks_label(matrix,feat):
    tmp = matrix.loc[:,feat]
    tmpmin = min(tmp)
    tmpmax = max(tmp)
    tmprange = tmpmax - tmpmin
    step = tmprange / 100
    return np.arange(tmpmin,tmpmax+step,step)

In [124]:
def labelticks(matrix,feat):
    tmp = matrix.loc[:,feat]
    tmpmin = min(tmp)
    tmpmax = max(tmp)
    tmprange = tmpmax - tmpmin
    step = tmprange / 20
    return np.arange(tmpmin,tmpmax+step,step)

In [125]:
def minmax_label(matrix,feat):
    tmp = matrix.loc[:,feat]
    tmpmin = min(tmp)
    tmpmax = max(tmp)
    return tmpmin,tmpmax

### Axis labels

In [126]:
def addlabels(canvas,ticks):
    width = canvas.width
    height = canvas.height
    margin = rectheight
    newheight = height + margin
    
    unicanvas = Image.new('RGBA',(width,newheight),(255,255,255,0))
    unicanvas.paste(canvas,(0,0))
    
    font = ImageFont.truetype('VeraMono.ttf', rectheight - 16 )
    
    # fontWidth, fontHeight = font.getsize(high) # can use if I have sizing issues

    draw = ImageDraw.Draw(unicanvas,'RGBA')
    
    for i in range(len(ticks)):
        xpos = i * rectwidth * 5
        draw.text((xpos,newheight - margin ),text=str(int(ticks[i])),font=font,fill=(0,0,0))
    
    return unicanvas

In [169]:
def scorenorm(item):
    scoremin = min(scores)
    scorerng = max(scores) - scoremin
    return int( ( ( 1 - ( ( item - scoremin ) / scorerng ) ) ) * 50 ) + 50

# test plot

In [170]:
def updown(X,cfeat,rectwidth,rectheight,pad):
    bins = binticks_label(test_matrix,cfeat)
    nbins = len(bins)
    pos = pd.cut(X[labels==1].loc[:,cfeat],bins=bins,labels=False,include_lowest=True)
    neg = pd.cut(X[labels==0].loc[:,cfeat],bins=bins,labels=False,include_lowest=True)
    px_w = ( nbins - 1 ) * rectwidth # a hack; won't work with all subsets
        
    ###########
    ### POS ###
    ###########
    
    posbinmax = pos.value_counts().max()
    overmax = int( math.ceil( neg.value_counts().max() * prior ) )
    
    if posbinmax >= overmax:
        ppx_h = posbinmax * rectheight
    else:
        ppx_h = overmax * rectheight
    
    up = Image.new('RGBA',(px_w,ppx_h),(255,255,255,0))
    updraw = ImageDraw.Draw(up)
        
    for binn in range(nbins):
        xpos = binn * rectwidth
        ypos = ppx_h - rectheight

        tmp = pos[pos==binn]
        
        for i in tmp.index:
            bbox = [(xpos+pad,ypos+pad),(xpos+rectwidth-pad,ypos+rectheight-pad)]
            if i in hotidxs:
                if i==singleton:
                    #updraw.rectangle(bbox,fill='hsl(0,50%,'+str(scorenorm(scores[i]))+'%)',outline='hsl(0,50%,'+str(scorenorm(scores[i]))+'%)')
                    updraw.rectangle(bbox,fill='hsl(0,0%,'+str(scorenorm(scores[i]))+'%)',outline='hsl(0,0%,0%)')
                    #updraw.rectangle(bbox,fill='hsl(0,50%,50%)',outline='hsl(0,50%,50%)') # red
                else:
                    #updraw.rectangle(bbox,fill='hsl(120,50%,'+str(scorenorm(scores[i]))+'%)',outline='hsl(120,50%,'+str(scorenorm(scores[i]))+'%)')
                    updraw.rectangle(bbox,fill='hsl(0,0%,'+str(scorenorm(scores[i]))+'%)',outline='hsl(0,0%,0%)')
                    #updraw.rectangle(bbox,fill='hsl(120,50%,50%)',outline='hsl(120,50%,50%)') # green
            else:
                if i==singleton:
                    #updraw.rectangle(bbox,fill='hsl(0,50%,'+str(scorenorm(scores[i]))+'%)',outline='hsl(0,50%,'+str(scorenorm(scores[i]))+'%)')
                    updraw.rectangle(bbox,fill='hsl(0,0%,'+str(scorenorm(scores[i]))+'%)',outline='hsl(0,0%,0%)')
                    #updraw.rectangle(bbox,fill='hsl(0,50%,50%)',outline='hsl(0,50%,50%)') # red
                else:
                    pass
                    updraw.rectangle(bbox,fill='hsl(0,0%,'+str(scorenorm(scores[i]))+'%)',outline='hsl(0,0%,0%)')
                    #updraw.rectangle(bbox,fill='hsl(0,0%,75%)',outline=None) # grey
            
            ypos = ypos - rectheight
        
        actual_positives = len(tmp)
        n = actual_positives + len(neg[neg==binn])
        expected_positives = int( n * float(prior) )
        lift = actual_positives - expected_positives
        
        if lift > 0:
            ypos = ppx_h - lift*rectheight
        else:
            ypos = ppx_h
            
        coords = [(xpos,ypos),(xpos+rectwidth,ypos)]
        #updraw.line(coords,fill=(220,101,113),width=6) # red

        try:
            coords = [(last_xpos+rectwidth,last_ypos),(xpos,ypos)]
            #updraw.line(coords,fill=(220,101,113),width=6)
        except:
            pass

        last_ypos = ypos
        last_xpos = xpos
        
    up = addlabels(up,labelticks(test_matrix,cfeat))    
    updraw = ImageDraw.Draw(up) # will use it below

    ###########
    ### NEG ###
    ###########
    
    # reset for line thing
    last_ypos = None
    last_xpos = None
    
    negbinmax = neg.value_counts().max()
    
    npx_h = negbinmax * rectheight
    
    down = Image.new('RGBA',(px_w,npx_h),(255,255,255,0))
    downdraw = ImageDraw.Draw(down)
    
    for binn in range(nbins):
        xpos = binn * rectwidth
        ypos = npx_h - rectheight
                
        tmp = neg[neg==binn]
        for i in tmp.index:
            bbox = [(xpos+pad,ypos+pad),(xpos+rectwidth-pad,ypos+rectheight-pad)]
            
            if i in hotidxs:
                if i==singleton:
                    #downdraw.rectangle(bbox,fill='hsl(0,50%,'+str(scorenorm(scores[i]))+'%)',outline='hsl(0,50%,'+str(scorenorm(scores[i]))+'%)')
                    downdraw.rectangle(bbox,fill='hsl(0,0%,'+str(scorenorm(scores[i]))+'%)',outline='hsl(0,0%,0%)')
                    #downdraw.rectangle(bbox,fill='hsl(0,50%,50%)',outline='hsl(0,50%,50%)') # red
                else:
                    #downdraw.rectangle(bbox,fill='hsl(120,50%,'+str(scorenorm(scores[i]))+'%)',outline='hsl(120,50%,'+str(scorenorm(scores[i]))+'%)')
                    downdraw.rectangle(bbox,fill='hsl(0,0%,'+str(scorenorm(scores[i]))+'%)',outline='hsl(0,0%,0%)')
                    #downdraw.rectangle(bbox,fill='hsl(120,50%,50%)',outline='hsl(120,50%,50%)') # green
            else:
                if i==singleton:
                    #downdraw.rectangle(bbox,fill='hsl(0,50%,'+str(scorenorm(scores[i]))+'%)',outline='hsl(0,50%,'+str(scorenorm(scores[i]))+'%)')
                    downdraw.rectangle(bbox,fill='hsl(0,0%,'+str(scorenorm(scores[i]))+'%)',outline='hsl(0,0%,0%)')
                    #downdraw.rectangle(bbox,fill='hsl(0,50%,50%)',outline='hsl(0,50%,50%)') # red
                else:
                    pass
                    downdraw.rectangle(bbox,fill='hsl(0,0%,'+str(scorenorm(scores[i]))+'%)',outline='hsl(0,0%,0%)')
                    #downdraw.rectangle(bbox,fill='hsl(0,0%,75%)',outline=None) # grey
            
            ypos = ypos - rectheight
               
        actual_positives = len(pos[pos==binn])
        n = actual_positives + len(tmp)
        expected_positives = int( n * float(prior) )
        lift = actual_positives - expected_positives        
                
        if lift < 0:
            ypos = npx_h - abs(lift)*rectheight
        else:
            ypos = npx_h
            
        coords = [(xpos,ypos),(xpos+rectwidth,ypos)]
        #downdraw.line(coords,fill=(112,159,210),width=6)
        
        try:
            coords = [(last_xpos+rectwidth,last_ypos),(xpos,ypos)]
            #downdraw.line(coords,fill=(112,159,210),width=6)
        except:
            pass
        
        negbinheight = len(tmp)
        nneg = len(neg)
        npos = len(pos)
        overheight = int( negbinheight * npos / nneg )               
        ypos = ppx_h - overheight*rectheight # ppx_h bc plotting to 'up'

        coords = [(xpos,ypos),(xpos+rectwidth,ypos)]
        #updraw.line(coords,fill=(255,166,77),width=6)
        
        try:
            coords = [(last_xpos+rectwidth,last_ypos),(xpos,ypos)]
            #updraw.line(coords,fill=(255,166,77),width=6)
        except:
            pass
        
        last_ypos = ypos
        last_xpos = xpos
    
    unicanvas = Image.new('RGBA',(px_w,up.height+npx_h),(255,255,255,0))
    unicanvas.paste(up,(0,0))

    down_flipped = down.transpose(Image.FLIP_TOP_BOTTOM)
    unicanvas.paste(down_flipped,(0,up.height))

    return unicanvas,up.height,px_w

In [171]:
pad = 0

In [172]:
rectwidth = 56
rectheight = 56

In [173]:
im,uph,px_w = updown(test_matrix,cfeat,rectwidth,rectheight,pad)

/Library/Python/2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Library/Python/2.7/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [174]:
im.save("/Users/damoncrockett/Desktop/"+cfeat+"_test"+"_hotonlyepsinglerisk50.png")

In [168]:
#halfheight = 1575/2
#cropbox = [0,uph-halfheight,px_w,uph+halfheight]
#im = im.crop(cropbox)